# K2TranPix 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.signal import convolve2d
from scipy.signal import deconvolve
from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import time as t
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)


def MinThrustframe(data,thrust):
    mean = np.nanmean(data[thrust+1],axis = 0)
    std = np.nanstd((data[thrust+1] - mean), axis = (1,2))
    Framemin = np.where(std == np.nanmin(abs(std)))[0][0]
    return thrust[Framemin]+1

def DriftKiller(data,thrust):
    # The right value choice here is a bit ambiguous, though it seems that typical variations are <10.
    Drift = (abs(data[thrust+1]-data[thrust-1]) < 10)*1.0 
    Drift[Drift == 0] = np.nan
    j = 0
    for i in range(len(thrust)):
        data[j:thrust[i]] = data[j:thrust[i]]*Drift[i]
        j = thrust[i]
    return data

def FindMinFrame(data):
    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(data[i:-n_steps+i:n_steps,:,:] - data[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
    return Framemin

def ObjectMask(datacube,Framemin):
    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    return Maskv2

def ThrustObjectMask(data,thrust):
    StartMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        Start = data[thrust[:3]+1]*StartMask/(np.nanmedian(data[thrust[:3]+1]*StartMask, axis = (1,2))+np.nanstd(data[thrust[:3]+1]*StartMask, axis = (1,2)))[:,None,None]
        Start = Start >= 1
        temp = (np.nansum(Start*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        StartMask = StartMask*temp


    EndMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        End = data[thrust[-3:]+1]*EndMask/(np.nanmedian(data[thrust[-3:]+1]*EndMask, axis = (1,2))+np.nanstd(data[thrust[-3:]+1]*EndMask, axis = (1,2)))[:,None,None]
        End = End >= 1
        temp = (np.nansum(End*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        EndMask = EndMask*temp
    
        
    Mask = np.nansum([np.ma.masked_invalid(StartMask).mask,np.ma.masked_invalid(EndMask).mask],axis=(0))*1.0
    Mask[Mask!=2] = 1
    Mask[Mask==2] = np.nan
    return Mask



def EventSplitter(events,Times,Masks,framemask):
    Events = []
    times = []
    mask = []
    for i in range(len(events)):
        # Check if there are multiple transients
        Coincident = Masks[events[i]]*framemask[events[i]]*1
        positions = np.where(Coincident == 1)
        if len(positions[0]) > 1:
            for p in range(len(positions[0])):
                eventmask = np.zeros((Masks.shape[1],Masks.shape[2]))
                eventmask[positions[0][p],positions[1][p]] = 1
                eventmask = convolve(eventmask,np.ones((3,3)),mode='constant', cval=0.0)
                Similar = np.where((Masks[Times[i][0]:,:,:]*eventmask == eventmask).all(axis=(1,2)))[0]
                
                if len((np.diff(Similar)<5)) > 1:
                    
                    if len(np.where((np.diff(Similar)<5) == False)[0]) > 0:
                        SimEnd = np.where((np.diff(Similar)<5) == False)[0][0] 
                    else:
                        SimEnd = -1
                else:
                    SimEnd = 0

                Similar = Similar[:SimEnd]
                if len(Similar) > 1:
                    timerange = [Similar[0]+Times[i][0]-1,Similar[-1]+Times[i][0]+1]
                    if len(timerange) > 1:
                        Events.append(events[i])
                        times.append(timerange)
                        mask.append(eventmask)
                
        else:
            Events.append(events[i])
            times.append(Times[i])
            mask.append(Masks[events[i]])
            

    return Events, times, mask

def Asteroid_fitter(Mask,Time,Data, plot = False):
    lc = np.nansum(Data*Mask,axis=(1,2))
    middle = np.where(np.nanmax(lc[Time[0]-1:Time[-1]+1]) == lc)[0][0]
    if abs(Time[0] - Time[1]) < 4:
        x = np.arange(middle-1,middle+1+1,1)
    else:
        x = np.arange(middle-2,middle+2+1,1)
    if x[-1] > len(lc) - 1:
        x = x[x<len(lc)]
    x2 = np.arange(0,len(x),1)
    y = lc[x]
    p1, residual, _, _, _ = np.polyfit(x,y,2, full = True)
    p2 = np.poly1d(p1)
    AvLeft = np.nansum(abs(lc[Time[0]:Time[-1]] - p2(np.arange(Time[0],Time[-1]))))/(Time[-1]-Time[0])
    maxpoly = np.where(np.nanmax(p2(x)) == p2(x))[0][0]
    if (AvLeft < 200) &  (abs(middle - x[maxpoly]) < 2):
        asteroid = True
        if plot == True:
            p2 = np.poly1d(p1)
            plt.figure()
            plt.plot(x,lc[x],'.')
            plt.plot(x,p2(x),'.')
            plt.ylabel('Counts')
            plt.xlabel('Time')
            plt.title('Residual = ' + str(residual))
            
    else:
        asteroid = False

    return asteroid

def Smoothmax(interval,Lightcurve,qual):
    x = np.arange(interval[0],interval[1],1.)
    x[qual[interval[0]:interval[-1]]!=0] = np.nan 
    nbins = int(len(x)/5)
    y = np.copy(Lightcurve[interval[0]:interval[-1]])
    y[qual[interval[0]:interval[-1]]!=0] = np.nan
    
    if np.nansum(x) > 0:
        n, _ = np.histogram(x, bins=nbins,range=(np.nanmin(x),np.nanmax(x)))
        sy, _ = np.histogram(x, bins=nbins, weights=y,range=(np.nanmin(x),np.nanmax(x)))
        sy2, _ = np.histogram(x, bins=nbins, weights=y*y,range=(np.nanmin(x),np.nanmax(x)))
        mean = sy / n
        std = np.sqrt(sy2/n - mean*mean)

        xrange = np.linspace(np.nanmin(x),np.nanmax(x),len(x))
        y_smooth = np.interp(xrange, (_[1:] + _[:-1])/2, mean)
        y_smooh_error = np.interp(xrange, (_[1:] + _[:-1])/2, std)

        temp = np.copy(y)
        temp[y_smooh_error>10] =np.nan

        maxpos = np.where(temp == np.nanmax(temp))[0]+interval[0]
    else:
        maxpos = 0
    return maxpos

def ThrusterElim(Events,Times,Masks,Firings,Quality,qual,Data):
    temp = []
    temp2 = []
    temp3 = []
    asteroid = []
    asttime = []
    astmask = []
    for i in range(len(Events)):
        Range = Times[i][-1] - Times[i][0]
        if (Range > 0) & (Range/Data.shape[0] < 0.8) & (Times[i][0] > 5): 
            begining = Firings[(Firings >= Times[i][0]-2) & (Firings <= Times[i][0]+1)]
            if len(begining) == 0:
                begining = Quality[(Quality == Times[i][0])] #& (Quality <= Times[i][0]+1)]
            end = Firings[(Firings >= Times[i][-1]-1) & (Firings <= Times[i][-1]+2)]
            if len(end) == 0:
                end = Quality[(Quality == Times[i][-1])] #& (Quality <= Times[i][-1]+1)]
            eventthrust = Firings[(Firings >= Times[i][0]) & (Firings <= Times[i][-1])]

            if (~begining.any() & ~end.any()) & (len(eventthrust) < 3):
                
                if Asteroid_fitter(Masks[i],Times[i],Data):
                    asteroid.append(Events[i])
                    asttime.append(Times[i])
                    astmask.append(Masks[i])
                else:
                    LC = np.nansum(Data[Times[i][0]:Times[i][-1]+3]*Masks[i], axis = (1,2))
                    if (np.where(np.nanmax(LC) == LC)[0] < Range).all():
                    
                        temp.append(Events[i])
                        temp2.append(Times[i])
                        temp3.append(Masks[i])

            elif len(eventthrust) >= 3:

                if begining.shape[0] == 0:
                    begining = 0
                else:
                    begining = begining[0]   
                if end.shape[0] == 0:
                    end = Times[i][-1] + 10
                else:
                    end = end[0]
                LC = np.nansum(Data*Masks[i], axis = (1,2))
                maxloc = Smoothmax(Times[i],LC,qual)

                if ((maxloc > begining).all() & (maxloc < end)).all(): 
                    premean = np.nanmean(LC[eventthrust-1]) 
                    poststd = np.nanstd(LC[eventthrust+1])
                    postmean = np.nanmedian(LC[eventthrust+1])
                    Outsidethrust = Firings[(Firings < Times[i][0]) | (Firings > Times[i][-1]+20)]
                    Outsidemean = np.nanmedian(LC[Outsidethrust+1])
                    Outsidestd = np.nanstd(LC[Outsidethrust+1])
                    if  postmean > Outsidemean+2*Outsidestd:
                        temp.append(Events[i])
                        temp2.append(Times[i])
                        temp3.append(Masks[i])


    events = np.array(temp)
    eventtime = np.array(temp2)
    eventmask = np.array(temp3)
    return events, eventtime, eventmask, asteroid, asttime, astmask


def pix2coord(x,y,mywcs):
    wx, wy = mywcs.wcs_pix2world(x, y, 0)
    return np.array([float(wx), float(wy)])

def Get_gal_lat(mywcs,datacube):
    ra, dec = mywcs.wcs_pix2world(int(datacube.shape[1]/2), int(datacube.shape[2]/2), 0)
    b = SkyCoord(ra=float(ra)*u.degree, dec=float(dec)*u.degree, frame='icrs').galactic.b.degree
    return b

def Save_space(Save,File):
    try:
        
        if not os.path.exists(Save+File.split('K2/')[-1]):
            os.makedirs(Save+File.split('K2/')[-1].split('ktwo')[0])
    except FileExistsError:
        pass

def K2TranPixFig(Events,Eventtime,Eventmask,Data,Time,Frames,wcs,Save,File,Quality,Thrusters,Framemin,Datacube):
    for i in range(len(Events)):
            # Check if there are multiple transients
            #Find Coords of transient
            position = np.where(Eventmask[i])

            maxcolor = np.nanmax(Data[Eventtime[i][0]:Eventtime[i][-1],(Eventmask[i]==1)])
            
            Mid = np.where(Data[Eventtime[i][0]:Eventtime[i][-1],(Eventmask[i]==1)] == maxcolor)
            
            Coord = pix2coord(Mid[1],Mid[0],wcs)
            # Generate a light curve from the transient masks
            LC = np.nansum(Data*Eventmask[i], axis = (1,2))
            BG = Data*~Frames[Events[i]]
            BG[BG <= 0] =np.nan
            BGLC = np.nanmedian(BG, axis = (1,2))
            
            Obj = np.ma.masked_invalid(Data[Framemin]).mask
            ObjLC = np.nansum(Datacube*Obj,axis = (1,2))
            ObjLC = ObjLC*np.nanmax(LC)/np.nanmax(ObjLC)
            
            
            fig = plt.figure(figsize=(10,6))
            # set up subplot grid
            gridspec.GridSpec(2,3)

            # large subplot
            plt.subplot2grid((2,3), (0,0), colspan=2, rowspan=2)
            plt.title('Event light curve (BJD '+str(round(Time[Eventtime[i][0]]-Time[0],2))+', RA '+str(round(Coord[0],3))+', DEC '+str(round(Coord[1],3))+')')
            plt.xlabel('Time (+'+str(Time[0])+' BJD)')
            plt.ylabel('Counts')
            plt.plot(Time - Time[0], LC,'.', label = 'Event LC')
            plt.plot(Time - Time[0], BGLC,'k.', label = 'Background LC')
            plt.plot(Time - Time[0], ObjLC,'kx', label = 'Scalled object LC')
            plt.axvspan(Time[Eventtime[i][0]]-Time[0],Time[Eventtime[i][-1]]-Time[0], color = 'orange', label = 'Event duration')
            plt.axvline(Time[Quality[0]]-Time[0],color = 'red', linestyle='dashed',label = 'Quality', alpha = 0.5)
            for j in range(Quality.shape[0]-1):
                j = j+1 
                plt.axvline(Time[Quality[j]]-Time[0], linestyle='dashed', color = 'red', alpha = 0.5)
            # plot Thurster firings 
            plt.axvline(Time[Thrusters[0]]-Time[0],color = 'red',label = 'Thruster', alpha = 0.5)
            for j in range(Thrusters.shape[0]-1):
                j = j+1 
                plt.axvline(Time[Thrusters[j]]-Time[0],color = 'red', alpha = 0.5)
            xmin = Time[Eventtime[i][0]]-Time[0]-(Eventtime[i][-1]-Eventtime[i][0])/10
            xmax = Time[Eventtime[i][-1]]-Time[0]+(Eventtime[i][-1]-Eventtime[i][0])/10
            if xmin < 0:
                xmin = 0
            if xmax > Time[-1] - Time[0]:
                xmax = Time[-1] - Time[0]
            plt.xlim(xmin,xmax) # originally 48 for some reason
            plt.ylim(0,np.nanmax(LC[Eventtime[i][0]:Eventtime[i][-1]])+0.1*np.nanmax(LC[Eventtime[i][0]:Eventtime[i][-1]]))
            plt.legend(loc = 1)
            # small subplot 1 Reference image plot
            plt.subplot2grid((2,3), (0,2))
            plt.title('Reference')
            plt.imshow(Data[Framemin,:,:], origin='lower',vmin=0,vmax = maxcolor)
            current_cmap = plt.cm.get_cmap()
            current_cmap.set_bad(color='black')
            plt.colorbar(fraction=0.046, pad=0.04)
            plt.plot(position[1],position[0],'r.',ms = 15)
            # small subplot 2 Image of event
            plt.subplot2grid((2,3), (1,2))
            plt.title('Event')
            plt.imshow(Data[np.where(Data*Eventmask[i]==np.nanmax(Data[Eventtime[i][0]:Eventtime[i][-1]]*Eventmask[i]))[0][0],:,:], origin='lower',vmin=0,vmax = maxcolor)
            current_cmap = plt.cm.get_cmap()
            current_cmap.set_bad(color='black')
            plt.colorbar(fraction=0.046, pad=0.04)
            plt.plot(position[1],position[0],'r.',ms = 15)
            # fit subplots and save fig
            fig.tight_layout()
            #fig.set_size_inches(w=11,h=7)
            Save_space(Save+'Figures/',File)
            
            plt.savefig(Save+'Figures/'+File.split('/')[-1].split('-')[0]+'_'+str(i)+'.pdf', bbox_inches = 'tight')
            plt.close;


import matplotlib.animation as animation
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def K2TranPixGif(Events,Eventtime,Eventmask,Data,wcs,Save,File):
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=1, metadata=dict(artist='Me'), bitrate=1800)
    for i in range(len(Events)):
        position = np.where(Eventmask[i])
        
        maxcolor = np.nanmax(Data[Eventtime[i][0]:Eventtime[i][-1],(Eventmask[i] == 1)])

        xmin = Eventtime[i][0]-(Eventtime[i][1]-Eventtime[i][0])
        xmax = Eventtime[i][1]+(Eventtime[i][1]-Eventtime[i][0])
        if xmin < 0:
            xmin = 0
        if xmax > len(Data):
            xmax = len(Data)-1
        Section = Data[int(xmin):int(xmax),:,:]
        fig = plt.figure()
        fig.set_size_inches(6,6)
        ims = []
        for j in range(Section.shape[0]):
            im = plt.imshow(Section[j], origin='lower',vmin = 0, vmax = maxcolor, animated=True)
            plt.plot(position[1],position[0],'r.',ms = 15)
            ims.append([im])
        plt.title(File.split('/')[-1].split('-')[0]+' Event # '+str(i))
        ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True, repeat = False)
        c = plt.colorbar(fraction=0.046, pad=0.04)
        c.set_label('Counts')
        
        Save_space(Save+'Figures/',File)
        ani.save(Save+'Figures/'+File.split('/')[-1].split('-')[0]+'_Event_#_'+str(i)+'.mp4',dpi=300)
        plt.close();




def K2TranPix(pixelfile,save): # More efficient in checking frames
    start = t.time()
    Save = save + pixelfile.split('-')[1].split('_')[0]
    try:
        hdu = fits.open(pixelfile)
        dat = hdu[1].data
        datacube = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
        if datacube.shape[1] > 1 and datacube.shape[2] > 1:
            time = dat["TIME"] + 2454833.0
            Qual = hdu[1].data.field('QUALITY')
            thrusters = np.where((Qual == 1048576) | (Qual == 1089568) | (Qual == 1056768) | (Qual == 1064960) | (Qual == 1081376) | (Qual == 10240) | (Qual == 32768) )[0]
            quality = np.where(Qual != 0)[0]
            #calculate the reference frame
            Framemin = FindMinFrame(datacube)
            # Apply object mask to data
            Mask = ThrustObjectMask(datacube,thrusters)
        
            Maskdata = datacube*Mask

            # Make a mask for the object to use as a test to eliminate very bad pointings
            obj = np.ma.masked_invalid(Mask).mask
            objmed = np.nanmedian(datacube[thrusters+1]*obj,axis=(0))
            objstd = np.nanstd(datacube[thrusters+1]*obj,axis=(0))
            Maskdata[(np.nansum(datacube*obj,axis=(1,2)) < np.nansum(objmed-3*objstd)),:,:] = np.nan

            framemask = np.zeros(Maskdata.shape)

            framemask = ((Maskdata/abs(np.nanmedian(Maskdata, axis = (0))+3*(np.nanstd(Maskdata, axis = (0))))) >= 1)
            framemask[:,np.where(Maskdata > 100000)[1],np.where(Maskdata > 100000)[2]] = 0

            # Identify if there is a sequence of consecutive or near consecutive frames that meet condtition 

            Eventmask_ref = (convolve(framemask,np.ones((1,3,3)),mode='constant', cval=0.0))*1
            Eventmask = np.copy(Eventmask_ref)
            Eventmask[~np.where((convolve(Eventmask_ref,np.ones((5,1,1)),mode='constant', cval=0.0) >= 4))[0]] = 0
            Eventmask[Qual!=0,:,:] = False
            Eventmask_ref[Qual!=0,:,:] = False

            Index = np.where(np.nansum(Eventmask*1, axis = (1,2))>0)[0]


            events = []
            eventtime = []
            while len(Index) > 1:

                similar = np.where(((Eventmask[Index[0]]*Eventmask_ref[Index[0]:]) == Eventmask[Index[0]]).all(axis = (1,2)))[0]+Index[0]

                if len((np.diff(similar)<5)) > 1:



                    if len(np.where((np.diff(similar)<5) == False)[0]) > 0:
                        simEnd = np.where((np.diff(similar)<5) == False)[0][0] 
                    else:
                        simEnd = -1
                else:
                    simEnd = 0
                if (simEnd > 0):
                    similar = similar[:simEnd]
                elif (simEnd == 0):
                    similar = np.array([similar[0]])

                if len(similar) > 1:

                    events.append(similar[0])
                    temp = [similar[0]-1,similar[-1]+1]
                    eventtime.append(temp)
                    temp = []
                template = Eventmask[Index[0]]
                for number in similar:
                    if (np.nansum(template*1-Eventmask[number]*1) == 0):
                        Index = np.delete(Index, np.where(Index == number)[0])

            events, eventtime, eventmask = EventSplitter(events,eventtime,Eventmask,framemask)  
        
            events = np.array(events)
            eventmask = np.array(eventmask)
            eventtime = np.array(eventtime)

            temp = []
            for i in range(len(events)):
                if len(np.where(datacube[eventtime[i][0]:eventtime[i][-1]]*eventmask[i] > 100000)[0]) == 0:
                    temp.append(i)
            eventtime = eventtime[temp]
            events = events[temp]
            eventmask = eventmask[temp]
            if len(eventmask) > 0:
                middle = (convolve(eventmask,np.ones((1,3,3))) == np.nanmax(convolve(eventmask,np.ones((1,3,3))))) & (convolve(eventmask,np.ones((1,3,3)),mode='constant', cval=0.0) == np.nanmax(convolve(eventmask,np.ones((1,3,3)),mode='constant', cval=0.0)))
                eventmask = eventmask*middle


            # Eliminate events that begin/end within 2 cadences of a thruster fire
            events, eventtime, eventmask = EventSplitter(events,eventtime,Eventmask,framemask)  
            events = np.array(events)
            eventmask = np.array(eventmask)
            if len(eventmask) > 0:
                middle = (convolve(eventmask,np.ones((1,3,3))) == np.nanmax(convolve(eventmask,np.ones((1,3,3))))) & (convolve(eventmask,np.ones((1,3,3)),mode='constant', cval=0.0) == np.nanmax(convolve(eventmask,np.ones((1,3,3)),mode='constant', cval=0.0)))
                eventmask = eventmask*middle


            # Eliminate events that begin/end within 2 cadences of a thruster fire
            events, eventtime, eventmask, asteroid, asttime, astmask = ThrusterElim(events,eventtime,eventmask,thrusters,quality,Qual,Maskdata)
            events = np.array(events)
            eventtime = np.array(eventtime)
            eventmask = np.array(eventmask)


            coincident = []
            i = 0
            while i < len(events):
                coincident = ((eventtime[:,0] >= eventtime[i,0]-1) & (eventtime[:,0] <= eventtime[i,0]+1) & (eventtime[:,1] >= eventtime[i,1]-1) & (eventtime[:,1] <= eventtime[i,1]+1))       
                if sum(coincident*1) > 1:
                    newmask = (np.nansum(eventmask[coincident],axis = (0)) > 0)*1 

                    events = np.delete(events,np.where(coincident)[0][1:])
                    eventtime = np.delete(eventtime,np.where(coincident)[0][1:], axis = (0))
                    eventmask = np.delete(eventmask,np.where(coincident)[0][1:], axis = (0))
                    eventmask[i] = newmask

                i +=1
            
            # Save asteroids
            ast = {}
            ast['File'] = pixelfile
            ast['Asteroids'] = asteroid
            ast['Time'] = asttime
            ast['Mask'] = astmask

            astsave = Save + '/Asteroid/' + pixelfile.split('ktwo')[-1].split('-')[0]+'_Asteroid.npy'
            Save_space(Save + '/Asteroid/',pixelfile)
            np.save(astsave,ast)
            
            
            # Create an array that saves the total area of mask and time. 
            # 1st col pixelfile, 2nd duration, 3rd col area, 4th col number of events, 5th 0 if in galaxy, 1 if outside
            Result = np.zeros(5)
            # Define the coordinate system 
            funny_keywords = {'1CTYP4': 'CTYPE1',
                              '2CTYP4': 'CTYPE2',
                              '1CRPX4': 'CRPIX1',
                              '2CRPX4': 'CRPIX2',
                              '1CRVL4': 'CRVAL1',
                              '2CRVL4': 'CRVAL2',
                              '1CUNI4': 'CUNIT1',
                              '2CUNI4': 'CUNIT2',
                              '1CDLT4': 'CDELT1',
                              '2CDLT4': 'CDELT2',
                              '11PC4': 'PC1_1',
                              '12PC4': 'PC1_2',
                              '21PC4': 'PC2_1',
                              '22PC4': 'PC2_2'}
            mywcs = {}
            for oldkey, newkey in funny_keywords.items():
                mywcs[newkey] = hdu[1].header[oldkey] 
            mywcs = WCS(mywcs)

            # Check if in the galaxy plane -20 < b < 20
            b = Get_gal_lat(mywcs,datacube)
            if (float(b) > -20) and (float(b) < 20):
                Result[4] = 0 
            else:
                Result[4] = 1
            
            # Print the figures

            K2TranPixFig(events,eventtime,eventmask,Maskdata,time,Eventmask,mywcs,Save,pixelfile,quality,thrusters,Framemin,datacube)
            K2TranPixGif(events,eventtime,eventmask,Maskdata,mywcs,Save,pixelfile)
            
            
            Result[0] = int(pixelfile.split('ktwo')[-1].split('-')[0])
            Result[1] = (time[-1] - time[0]) - 3*len(thrusters)/48 # Last term is for removing coincident times
            Result[2] = np.nansum(Mask*1)
            Result[3] = 1*len(events)
            
        else:
            Result = np.ones(5)*np.nan
    except (OSError):
        Result = np.ones(5)*-1
    
    stop = t.time()
    
    print('Beep')
    print('Time taken=%f' %(stop-start))

In [4]:
path = '/Volumes/TOSHIBA EXt/K2/c06/*/*/'
save = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/c06/K2tranPix/'

Files = np.asarray(glob(path+'*.gz'))

In [5]:
Files.shape

(47560,)

In [ ]:
'/Volumes/TOSHIBA EXt/K2/c06/212500000/04000/ktwo212504061-c06_lpd-targ.fits.gz'

In [56]:
Parallel(n_jobs=4)(delayed(K2TranPix)(i,save) for i in Files)

Beep
Time taken=0.430392
Beep
Time taken=0.561284
Beep
Time taken=0.668057
Beep
Time taken=2.295675
Beep
Time taken=2.772301
Beep
Time taken=3.753604
Beep
Time taken=0.847589
Beep
Time taken=0.464510
Beep
Time taken=0.862802
Beep
Time taken=1.867960
Beep
Time taken=0.665924
Beep
Time taken=0.731643
Beep
Time taken=0.676108
Beep
Time taken=4.364553
Beep
Time taken=0.579201
Beep
Time taken=8.093565
Beep
Time taken=0.933354
Beep
Beep
Time taken=0.988226
Time taken=0.593913
Beep
Beep
Time taken=2.201056
Time taken=0.450929
Beep
Time taken=1.591014
Beep
Time taken=0.591070
Beep
Time taken=0.984209
Beep
Time taken=0.615064
Beep
Time taken=0.879591
Beep
Time taken=0.653045
Beep
Time taken=0.791476
Beep
Time taken=0.774867
Beep
Time taken=3.214541
Beep
Time taken=1.056757
Beep
Time taken=1.309230
Beep
Time taken=0.725061
Beep
Time taken=0.643959
Beep
Time taken=0.694191
Beep
Time taken=0.694387
Beep
Time taken=0.755398
Beep
Time taken=0.753996
Beep
Time taken=5.154960
Beep
Time taken=1.228961


Beep
Time taken=1.764529
Beep
Time taken=2.056333
Beep
Time taken=1.420650
Beep
Time taken=1.865424
Beep
Time taken=0.625958
Beep
Time taken=11.277247
Beep
Time taken=1.821397
Beep
Time taken=3.708783
Beep
Time taken=1.101416
Beep
Time taken=0.547524
Beep
Time taken=0.647999
Beep
Time taken=0.624181
Beep
Time taken=1.389998
Beep
Time taken=0.939047
Beep
Time taken=1.549877
Beep
Time taken=0.649500
Beep
Time taken=0.602970
Beep
Time taken=2.449082
Beep
Time taken=1.256761
Beep
Time taken=1.089230
Beep
Time taken=1.652743
Beep
Time taken=1.515913
Beep
Time taken=1.683128
Beep
Time taken=1.146530
Beep
Time taken=1.285649
Beep
Time taken=2.258152
Beep
Time taken=19.854915
Beep
Time taken=2.467019
Beep
Time taken=2.614618
Beep
Time taken=1.021477
Beep
Time taken=1.212474
Beep
Time taken=1.172624
Beep
Time taken=2.240776
Beep
Time taken=0.798776
Beep
Time taken=1.165669
Beep
Time taken=0.906527
Beep
Time taken=0.670648
Beep
Time taken=0.671387
Beep
Time taken=5.891171
Beep
Time taken=1.77125

Beep
Time taken=1.111570
Beep
Time taken=1.361142
Beep
Time taken=1.299942
Beep
Time taken=1.368388
Beep
Time taken=3.314880
Beep
Time taken=0.973745
Beep
Time taken=2.563556
Beep
Time taken=1.130107
Beep
Time taken=0.703955
Beep
Time taken=1.464172
Beep
Time taken=7.450243
Beep
Time taken=5.631508
Beep
Time taken=1.969744
Beep
Time taken=0.783028
Beep
Time taken=0.957834
Beep
Time taken=1.099699
Beep
Time taken=1.278636
Beep
Time taken=0.889126
Beep
Time taken=1.049808
Beep
Time taken=0.888098
Beep
Time taken=0.949943
Beep
Time taken=1.243425
Beep
Time taken=0.652778
Beep
Time taken=0.960343
Beep
Time taken=1.141589
Beep
Time taken=1.327849
Beep
Time taken=0.982984
Beep
Time taken=1.032120
Beep
Time taken=1.369005
Beep
Time taken=1.009255
Beep
Time taken=1.513008
Beep
Time taken=28.463100
Beep
Time taken=1.517656
Beep
Time taken=2.002885
Beep
Time taken=4.588670
Beep
Time taken=11.360655
Beep
Time taken=1.046408
Beep
Time taken=2.287857
Beep
Time taken=3.314105
Beep
Time taken=1.20500

Beep
Time taken=1.263859
Beep
Time taken=0.794542
Beep
Time taken=0.999381
Beep
Time taken=0.922095
Beep
Time taken=0.846344
Beep
Time taken=0.905410
Beep
Time taken=5.621915
Beep
Time taken=0.703722
Beep
Time taken=1.065818
Beep
Time taken=1.565284
Beep
Time taken=0.719028
Beep
Time taken=1.118779
Beep
Time taken=0.959336
Beep
Time taken=3.592117
Beep
Time taken=29.261196
Beep
Time taken=1.379296
Beep
Time taken=2.929822
Beep
Time taken=0.769253
Beep
Time taken=1.489384
Beep
Time taken=1.449325
Beep
Time taken=1.417258
Beep
Time taken=1.036418
Beep
Time taken=0.831885
Beep
Time taken=1.747021
Beep
Time taken=1.494061
Beep
Time taken=2.179185
Beep
Time taken=0.964514
Beep
Time taken=1.108559
Beep
Time taken=0.794916
Beep
Time taken=8.799849
Beep
Time taken=29.365413
Beep
Time taken=2.528694
Beep
Time taken=0.995400
Beep
Time taken=0.859454
Beep
Time taken=1.114398
Beep
Time taken=1.096302
Beep
Time taken=0.758100
Beep
Time taken=1.156264
Beep
Time taken=1.881902
Beep
Time taken=0.81278

Beep
Time taken=0.959122
Beep
Time taken=1.126035
Beep
Time taken=0.679439
Beep
Time taken=0.833459
Beep
Time taken=1.109207
Beep
Time taken=1.095238
Beep
Time taken=0.765208
Beep
Time taken=1.538479
Beep
Time taken=1.295523
Beep
Time taken=1.065580
Beep
Time taken=1.287067
Beep
Time taken=1.376342
Beep
Time taken=1.056377
Beep
Time taken=0.925018
Beep
Time taken=1.535529
Beep
Time taken=0.802810
Beep
Time taken=1.120928
Beep
Time taken=1.329685
Beep
Time taken=1.040805
Beep
Time taken=0.854359
Beep
Time taken=25.563151
Beep
Time taken=1.404989
Beep
Time taken=1.804672
Beep
Time taken=1.184237
Beep
Time taken=3.055612
Beep
Time taken=0.826312
Beep
Time taken=2.002741
Beep
Time taken=1.732724
Beep
Time taken=0.912739
Beep
Time taken=2.337090
Beep
Time taken=1.391449
Beep
Time taken=0.941531
Beep
Time taken=1.100882
Beep
Time taken=2.617125
Beep
Time taken=1.193047
Beep
Time taken=0.839038
Beep
Time taken=1.968277
Beep
Time taken=13.103843
Beep
Time taken=2.109706
Beep
Time taken=1.25772

Beep
Time taken=1.071128
Beep
Time taken=1.254566
Beep
Time taken=1.261622
Beep
Time taken=1.447562
Beep
Time taken=0.677045
Beep
Time taken=1.751560
Beep
Time taken=2.631839
Beep
Time taken=1.048977
Beep
Time taken=1.793823
Beep
Time taken=0.829134
Beep
Time taken=1.140981
Beep
Time taken=0.758456
Beep
Time taken=1.099053
Beep
Time taken=0.771322
Beep
Time taken=0.776828
Beep
Time taken=0.629069
Beep
Time taken=1.628220
Beep
Time taken=0.983990
Beep
Time taken=1.863125
Beep
Time taken=1.296148
Beep
Time taken=2.006872
Beep
Time taken=0.917522
Beep
Time taken=0.785151
Beep
Time taken=0.822638
Beep
Time taken=1.087501
Beep
Time taken=1.405768
Beep
Time taken=58.306959
Beep
Time taken=1.313919
Beep
Time taken=0.738035
Beep
Time taken=0.897923
Beep
Time taken=1.546314
Beep
Time taken=0.770760
Beep
Time taken=0.988144
Beep
Time taken=0.965167
Beep
Time taken=0.995962
Beep
Time taken=1.036406
Beep
Time taken=1.505659
Beep
Time taken=1.258100
Beep
Time taken=0.690974
Beep
Time taken=2.867163

Time taken=1.134042
Beep
Time taken=2.026965
Beep
Time taken=0.798499
Beep
Time taken=4.105869
Beep
Time taken=1.372468
Beep
Time taken=0.837268
Beep
Time taken=1.826799
Beep
Time taken=0.786915
Beep
Time taken=1.310415
Beep
Time taken=4.762684
Beep
Time taken=7.901306
Beep
Time taken=1.350915
Beep
Time taken=1.170776
Beep
Time taken=3.900371
Beep
Time taken=1.070276
Beep
Time taken=4.342045
Beep
Time taken=1.163153
Beep
Time taken=2.653010
Beep
Time taken=2.180798
Beep
Time taken=1.053830
Beep
Time taken=4.227909
Beep
Time taken=0.902376
Beep
Time taken=1.621240
Beep
Time taken=0.991765
Beep
Time taken=1.536841
Beep
Time taken=0.983263
Beep
Time taken=1.334058
Beep
Time taken=3.424940
Beep
Time taken=1.732880
Beep
Time taken=0.967757
Beep
Time taken=0.864677
Beep
Time taken=0.753915
Beep
Time taken=1.525592
Beep
Time taken=2.474596
Beep
Time taken=2.870397
Beep
Time taken=1.421334
Beep
Time taken=0.724102
Beep
Time taken=1.270806
Beep
Time taken=4.625042
Beep
Time taken=4.915652
Beep


Beep
Time taken=2.075854
Beep
Time taken=2.423209
Beep
Time taken=3.425862
Beep
Time taken=1.101714
Beep
Time taken=2.167776
Beep
Time taken=1.322065
Beep
Time taken=2.759965
Beep
Beep
Time taken=1.021076
Time taken=10.892959
Beep
Time taken=1.154272
Beep
Time taken=1.758621
Beep
Time taken=0.760959
Beep
Time taken=1.060144
Beep
Time taken=2.645802
Beep
Time taken=1.093170
Beep
Time taken=1.501991
Beep
Time taken=0.873559
Beep
Time taken=0.870871
Beep
Time taken=1.228590
Beep
Time taken=0.901467
Beep
Time taken=1.003053
Beep
Time taken=0.807115
Beep
Time taken=0.979320
Beep
Time taken=0.864865
Beep
Time taken=0.952174
Beep
Time taken=0.688537
Beep
Time taken=2.461682
Beep
Time taken=5.109340
Beep
Time taken=0.952875
Beep
Time taken=1.381122
Beep
Time taken=0.971518
Beep
Time taken=1.737246
Beep
Time taken=2.256109
Beep
Time taken=3.078698
Beep
Time taken=0.786181
Beep
Time taken=2.331315
Beep
Time taken=1.485892
Beep
Time taken=0.885789
Beep
Time taken=2.753442
Beep
Time taken=1.475092

Time taken=0.628488
Beep
Time taken=0.757056
Beep
Time taken=0.632733
Beep
Time taken=1.129232
Beep
Time taken=1.754168
Beep
Time taken=1.274720
Beep
Time taken=0.702990
Beep
Time taken=8.740408
Beep
Time taken=0.602831
Beep
Time taken=0.504690
Beep
Time taken=1.265374
Beep
Time taken=1.062646
Beep
Time taken=1.290238
Beep
Time taken=1.823732
Beep
Time taken=0.787974
Beep
Time taken=2.352341
Beep
Time taken=1.287508
Beep
Time taken=1.326677
Beep
Time taken=1.791469
Beep
Time taken=1.743261
Beep
Time taken=1.052604
Beep
Time taken=2.920860
Beep
Time taken=7.100180
Beep
Time taken=0.996950
Beep
Time taken=0.852707
Beep
Time taken=5.329700
Beep
Time taken=1.669408
Beep
Time taken=1.098429
Beep
Time taken=1.894485
Beep
Time taken=0.682389
Beep
Time taken=1.599933
Beep
Time taken=1.637970
Beep
Time taken=0.709063
Beep
Time taken=1.091366
Beep
Time taken=2.103068
Beep
Time taken=0.785922
Beep
Time taken=1.600534
Beep
Time taken=1.197102
Beep
Time taken=0.725756
Beep
Time taken=0.714010
Beep


KeyboardInterrupt: 

In [38]:
Rez = []
rez = []
for filename in tqdm_notebook(Files):
    rez = K2tranPix(filename,save)
    Rez.append(rez)
    rez = []
np.save(save+'c6_Results.npy',Rez)

HBox(children=(IntProgress(value=0, max=47560), HTML(value='')))

KeyboardInterrupt: 

In [33]:
i

NameError: name 'i' is not defined

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.signal import convolve2d
from scipy.signal import deconvolve
from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

def MinThrustframe(data,thrust):
    mean = np.nanmean(data[thrust+1],axis = 0)
    std = np.nanstd((data[thrust+1] - mean), axis = (1,2))
    Framemin = np.where(std == np.nanmin(abs(std)))[0][0]
    return thrust[Framemin]+1

def DriftKiller(data,thrust):
    # The right value choice here is a bit ambiguous, though it seems that typical variations are <10.
    Drift = (abs(data[thrust+1]-data[thrust-1]) < 10)*1.0 
    Drift[Drift == 0] = np.nan
    j = 0
    for i in range(len(thrust)):
        data[j:thrust[i]] = data[j:thrust[i]]*Drift[i]
        j = thrust[i]
    return data

def FindMinFrame(data):
    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(data[i:-n_steps+i:n_steps,:,:] - data[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
    return Framemin

def ObjectMask(datacube,Framemin):
    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    return Maskv2

def ThrustObjectMask(data,thrust):
    StartMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        Start = data[thrust[:3]+1]*StartMask/(np.nanmedian(data[thrust[:3]+1]*StartMask, axis = (1,2))+np.nanstd(data[thrust[:3]+1]*StartMask, axis = (1,2)))[:,None,None]
        Start = Start >= 1
        temp = (np.nansum(Start*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        StartMask = StartMask*temp


    EndMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        End = data[thrust[-3:]+1]*EndMask/(np.nanmedian(data[thrust[-3:]+1]*EndMask, axis = (1,2))+np.nanstd(data[thrust[-3:]+1]*EndMask, axis = (1,2)))[:,None,None]
        End = End >= 1
        temp = (np.nansum(End*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        EndMask = EndMask*temp
    
        
    Mask = np.nansum([np.ma.masked_invalid(StartMask).mask,np.ma.masked_invalid(EndMask).mask],axis=(0))*1.0
    Mask[Mask!=2] = 1
    Mask[Mask==2] = np.nan
    return Mask



def EventSplitter(events,Times,Masks,framemask):
    Events = []
    times = []
    mask = []
    for i in range(len(events)):
        # Check if there are multiple transients
        Coincident = Masks[events[i]]*framemask[events[i]]*1
        positions = np.where(Coincident == 1)
        if len(positions[0]) >1:
            for p in range(len(positions[0])):
                eventmask = np.zeros((Masks.shape[1],Masks.shape[2]))
                eventmask[positions[0][p],positions[1][p]] = 1
                eventmask = convolve(eventmask,np.ones((3,3)),mode='constant', cval=0.0)
                Similar = np.where((Masks[Times[i][0]:,:,:]*eventmask == eventmask).all(axis=(1,2)))[0]
                
                if len((np.diff(Similar)<5)) > 1:
                    
                    if len(np.where((np.diff(Similar)<5) == False)[0]) > 0:
                        SimEnd = np.where((np.diff(Similar)<5) == False)[0][0] 
                    else:
                        SimEnd = -1
                else:
                    SimEnd = 0
                #if len(np.where((np.diff(Similar)<5) == False)[0]) < 1:
                #    SimEnd = -1
                #else:
                #    SimEnd = np.where((np.diff(Similar)<5) == False)[0][0] - 1
                Similar = Similar[:SimEnd]
                if len(Similar) > 1:
                    timerange = [Similar[0]+Times[i][0]-1,Similar[-1]+Times[i][0]+1]
                    if len(timerange) > 1:
                        Events.append(events[i])
                        times.append(timerange)
                        mask.append(eventmask)
                
        else:
            Events.append(events[i])
            times.append(Times[i])
            mask.append(Masks[events[i]])
            

    return Events, times, mask

def Asteroid_fitter(Mask,Time,Data, plot = False):
    lc = np.nansum(Data*Mask,axis=(1,2))
    middle = np.where(np.nanmax(lc[Time[0]-1:Time[-1]+1]) == lc)[0][0]
    x = np.arange(middle-2,middle+2+1,1)
    x2 = np.arange(0,len(x),1)
    y = lc[[np.arange(middle-2,middle+2+1,1)]]
    p1, residual, _, _, _ = np.polyfit(x,y,2, full = True)
    p2 = np.poly1d(p1)
    AvLeft = np.nansum(abs(lc[Time[0]:Time[-1]] - p2(np.arange(Time[0],Time[-1]))))/(Time[-1]-Time[0])
    maxpoly = np.where(np.nanmax(p2(x)) == p2(x))[0][0]
    if (AvLeft < 100) &  (abs(middle - x[maxpoly]) < 2):
        asteroid = True
        if plot == True:
            p2 = np.poly1d(p1)
            plt.figure()
            plt.plot(x,lc[x],'.')
            plt.plot(x,p2(x),'.')
            plt.ylabel('Counts')
            plt.xlabel('Time')
            plt.title('Residual = ' + str(residual))
            
    else:
        asteroid = False

        
    return asteroid 

def Smoothmax(interval,Lightcurve,qual):
    x = np.arange(interval[0],interval[1],1.)
    x[qual[interval[0]:interval[-1]]!=0] = np.nan 
    nbins = int(len(x)/5)
    y = np.copy(Lightcurve[interval[0]:interval[-1]])
    y[qual[interval[0]:interval[-1]]!=0] = np.nan
    
    if np.nansum(x) > 0:
        n, _ = np.histogram(x, bins=nbins,range=(np.nanmin(x),np.nanmax(x)))
        sy, _ = np.histogram(x, bins=nbins, weights=y,range=(np.nanmin(x),np.nanmax(x)))
        sy2, _ = np.histogram(x, bins=nbins, weights=y*y,range=(np.nanmin(x),np.nanmax(x)))
        mean = sy / n
        std = np.sqrt(sy2/n - mean*mean)

        xrange = np.linspace(np.nanmin(x),np.nanmax(x),len(x))
        y_smooth = np.interp(xrange, (_[1:] + _[:-1])/2, mean)
        y_smooh_error = np.interp(xrange, (_[1:] + _[:-1])/2, std)

        temp = np.copy(y)
        temp[y_smooh_error>10] =np.nan

        maxpos = np.where(temp == np.nanmax(temp))[0]+interval[0]
    else:
        maxpos = 0
    return maxpos

def ThrusterElim(Events,Times,Masks,Firings,Quality,qual,Data):
    temp = []
    temp2 = []
    temp3 = []
    asteroid = []
    asttime = []
    astmask = []
    for i in range(len(Events)):
        Range = Times[i][-1] - Times[i][0]
        if (Range > 0) & (Range/Data.shape[0] < 0.8) & (Times[i][0] > 5): 
            begining = Firings[(Firings >= Times[i][0]-3) & (Firings <= Times[i][0]+1)]
            if len(begining) == 0:
                begining = Quality[(Quality >= Times[i][0]-3) & (Quality <= Times[i][0]+1)]
            end = Firings[(Firings >= Times[i][-1]-1) & (Firings <= Times[i][-1]+3)]
            if len(end) == 0:
                end = Quality[(Quality >= Times[i][-1]-1) & (Quality <= Times[i][-1]+3)]
            eventthrust = Firings[(Firings >= Times[i][0]) & (Firings <= Times[i][-1])]
            
            if (~begining.any() & ~end.any()) & (len(eventthrust) < 3):
                
                if Asteroid_fitter(Masks[i],Times[i],Data):
                    asteroid.append(Events[i])
                    asttime.append(Times[i])
                    astmask.append(Masks[i])
                else:
                    temp.append(Events[i])
                    temp2.append(Times[i])
                    temp3.append(Masks[i])
            elif len(eventthrust) >= 3:
                if begining.shape[0] == 0:
                    begining = 0
                else:
                    begining = begining[0]   
                if end.shape[0] == 0:
                    end = Times[i][-1] + 10
                else:
                    end = end[0]
                LC = np.nansum(Data*Masks[i], axis = (1,2))
                maxloc = Smoothmax(Times[i],LC,qual)

                if ((maxloc > begining) & (maxloc < end)): 
                    postmean = np.nanmedian(LC[eventthrust[(qual[eventthrust+1]==0)]+1])
                    Outsidethrust = Firings[(Firings < Times[i][0]) | (Firings > Times[i][-1]+20)]
                    Outsidemean = np.nanmedian(LC[(Outsidethrust[(qual[Outsidethrust+1]==0)]+1)])
                    Outsidestd = np.nanstd(LC[(Outsidethrust[(qual[Outsidethrust+1]==0)]+1)])
                    if  postmean > Outsidemean+Outsidestd:
                        temp.append(Events[i])
                        temp2.append(Times[i])
                        temp3.append(Masks[i])

    events = np.array(temp)
    eventtime = np.array(temp2)
    eventmask = np.array(temp3)
    return events, eventtime, eventmask, asteroid, asttime, astmask


def pix2coord(x,y,mywcs):
    wx, wy = mywcs.wcs_pix2world(x, y, 0)
    return np.array([float(wx), float(wy)])

def Get_gal_lat(mywcs,datacube):
    ra, dec = mywcs.wcs_pix2world(int(datacube.shape[1]/2), int(datacube.shape[2]/2), 0)
    b = SkyCoord(ra=float(ra)*u.degree, dec=float(dec)*u.degree, frame='icrs').galactic.b.degree
    return b




def K2tranPix(pixelfile,save): # More efficient in checking frames
    try:
        hdu = fits.open(pixelfile)
        dat = hdu[1].data
        datacube = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data#np.copy(testdata)#
        if datacube.shape[1] > 1 and datacube.shape[2] > 1:
            #print('in')
            time = dat["TIME"] + 2454833.0
            Qual = hdu[1].data.field('QUALITY')
            thrusters = np.where((Qual == 1048576) | (Qual == 1089568) | (Qual == 1056768) | (Qual == 1064960) | (Qual == 1081376) | (Qual == 10240) | (Qual == 32768) )[0]
            quality = np.where(Qual != 0)[0]
            #calculate the reference frame
            Framemin = FindMinFrame(datacube)
            # Apply object mask to data
            Mask = ThrustObjectMask(datacube,thrusters)

            Maskdata = datacube*Mask
            #Maskdata[Maskdata<0] = 0 

            #Motion control
            #Conv = convolve2d(np.ma.masked_invalid(Mask).mask, np.ones((3,3)), mode="same")
            #Maskdata = Maskdata*(Conv < 2)

            # Make a mask for the object to use as a test to eliminate very bad pointings
            obj = np.ma.masked_invalid(Mask).mask
            objmed = np.nanmedian(datacube[thrusters+1]*obj,axis=(0))
            objstd = np.nanstd(datacube[thrusters+1]*obj,axis=(0))
            Maskdata[(np.nansum(datacube*obj,axis=(1,2)) < np.nansum(objmed-3*objstd)),:,:] = np.nan

            #overflow = ((convolve(Maskdata[(np.nansum(datacube*obj,axis=(1,2)) > np.nansum(objmed+2*objstd)),:,:],
                                  #np.ones((1,1,2)), mode='constant', cval=0.0)) >= 1)*1.0
            #overflow[overflow == 1] = np.nan
            #overflow[overflow == 0] = 1
            #Maskdata[(np.nansum(datacube*obj,axis=(1,2)) > np.nansum(objmed+2*objstd)),:,:] = overflow
            #Maskdata[Maskdata > 170000] = np.nan
            #Stdframe = np.ones(Maskdata.shape)
            framemask = np.zeros(Maskdata.shape)

            #Index = (np.nansum(datacube*obj,axis=(1,2))>np.nansum(objmed-3*objstd)) #((np.nanstd(Maskdata,axis=(1,2)) > np.nanmedian(stddist)) & ((Maskdata.shape[1]>1) & (Maskdata.shape[2]>1))) 
            #framemask[Index] = (Maskdata[Index]/(np.nanmedian(Maskdata[Index])+2*(np.nanstd(Maskdata[Index])))) >= 1
            framemask = ((Maskdata/abs(np.nanmedian(Maskdata, axis = (0))+3*(np.nanstd(Maskdata, axis = (0))))) >= 1)
            framemask[:,np.where(Maskdata > 170000)[1],np.where(Maskdata > 170000)[2]] = 0
            #Index = ((np.nanstd(Maskdata) > np.nanmedian(stddist)+np.nanstd(stddist)) & ((Maskdata.shape[1]==1) | (Maskdata.shape[2] == 1))) 

            # Identify if there is a sequence of consecutive or near consecutive frames that meet condtition 
            #Eventmask = (convolve(framemask,np.ones((5,3,3)),mode='constant', cval=0.0) >= 3)


            Eventmask = (convolve(framemask,np.ones((1,3,3)),mode='constant', cval=0.0))*1
            Eventmask[~np.where((convolve(Eventmask,np.ones((5,1,1)),mode='constant', cval=0.0) >= 4))[0]] = 0
            Eventmask[Qual!=0,:,:] = False
            #print(np.where(np.nansum(Real_position*Eventmask,axis=(1,2))>0)[0])

            #Eventmask = DriftKiller(Eventmask*Maskdata,thrusters) > 0
            #Eventmask[np.isnan(Eventmask)] = 0
            Index = np.where(np.nansum(Eventmask*1, axis = (1,2))>0)[0]
            events = []
            eventtime = []
            while len(Index) > 1:
                #print('1')
                similar = Index[((Eventmask[Index[0]]*Eventmask[Index]) == Eventmask[Index[0]]).all(axis = (1,2))]
                
                #similar = similar[np.append(True, np.diff(similar)<5)]
                if len((np.diff(similar)<5)) > 1:
                    #print('2')
                    
                    if len(np.where((np.diff(similar)<5) == False)[0]) > 0:
                        simEnd = np.where((np.diff(similar)<5) == False)[0][0] 
                    else:
                        simEnd = -1
                else:
                    simEnd = 0

                if simEnd > 0:
                    similar = similar[:simEnd]
                else:
                    similar = np.array([similar[0]])

                if len(similar) > 1:
                    events.append(similar[0])
                    temp = [similar[0]-1,similar[-1]+1]
                    eventtime.append(temp)
                    temp = []
                for number in similar:
                    if ((Eventmask[Index[0]]*Eventmask[number]) == Eventmask[number]).all():
                        Index = np.delete(Index, np.where(Index == number)[0])

            events, eventtime, eventmask = EventSplitter(events,eventtime,Eventmask,framemask)     
            #eventtime = np.array(eventtime)
            events = np.array(events)
            eventmask = np.array(eventmask)



            # Eliminate events that begin/end within 2 cadences of a thruster fire

            events, eventtime, eventmask, asteroid, asttime, astmask = ThrusterElim(events,eventtime,eventmask,thrusters,quality,Qual,Maskdata)
            events = np.array(events)
            eventtime = np.array(eventtime)
            eventmask = np.array(eventmask)


            temp = []
            temp2 = []
            temp3 = []
            for i in range(len(eventtime)):
                if len(eventtime[i])>0:
                    t = np.nansum(Eventmask[eventtime[i][0]:eventtime[i][-1],:,:]*1,axis=(1,2)) > 0
                    if np.sum(t)/t.shape[0] > 0.6:
                        temp.append(eventtime[i][:])
                        temp2.append(events[i])
                        temp3.append(eventmask[i])
            eventtime = np.array(temp)
            events = np.array(temp2)
            eventmask = np.array(temp3)


            temp = []
            for i in range(len(events)):
                if len(np.where(datacube[eventtime[i,0]:eventtime[i,-1]]*eventmask[i] > 170000)[0]) == 0:
                    temp.append(i)
            eventtime = eventtime[temp]
            events = events[temp]
            eventmask = eventmask[temp]
            
            # Save asteroids
            ast = {}
            ast['File'] = pixelfile
            ast['Asteroids'] = asteroid
            ast['Time'] = asttime
            ast['Mask'] = astmask

            astsave = '/Users/ryanr/Documents/PhD/coding/Kepler/K2/' + pixelfile.split('-')[1].split('_')[0] + '/Asteroid/' + pixelfile.split('ktwo')[-1].split('-')[0]+'_Asteroid.npy'
            np.save(astsave,ast)
            
            
            # Create an array that saves the total area of mask and time. 
            # 1st col pixelfile, 2nd duration, 3rd col area, 4th col number of events, 5th 0 if in galaxy, 1 if outside
            Result = np.zeros(5)
            # Define the coordinate system 
            funny_keywords = {'1CTYP4': 'CTYPE1',
                              '2CTYP4': 'CTYPE2',
                              '1CRPX4': 'CRPIX1',
                              '2CRPX4': 'CRPIX2',
                              '1CRVL4': 'CRVAL1',
                              '2CRVL4': 'CRVAL2',
                              '1CUNI4': 'CUNIT1',
                              '2CUNI4': 'CUNIT2',
                              '1CDLT4': 'CDELT1',
                              '2CDLT4': 'CDELT2',
                              '11PC4': 'PC1_1',
                              '12PC4': 'PC1_2',
                              '21PC4': 'PC2_1',
                              '22PC4': 'PC2_2'}
            mywcs = {}
            for oldkey, newkey in funny_keywords.items():
                mywcs[newkey] = hdu[1].header[oldkey] 
            mywcs = WCS(mywcs)

            # Check if in the galaxy plane -20 < b < 20
            b = Get_gal_lat(mywcs,datacube)
            if (float(b) > -20) and (float(b) < 20):
                Result[4] = 0 
            else:
                Result[4] = 1
            # Check through each of the identified frame sets to see if there is anything and save a figure if so

            counter = 0
            for i in range(len(events)):
                # Check if there are multiple transients
                #Find Coords of transient
                position = np.where(eventmask[i])
                middle = int(np.ceil(np.nansum(eventmask[i])/2))
                Coord = pix2coord(position[0][middle],position[1][middle],mywcs)

                # Generate a light curve from the transient masks
                LC = np.nansum(Maskdata*eventmask[i], axis = (1,2))
                BG = Maskdata*~Eventmask[events[i]]
                BG[BG <= 0] =np.nan
                BGLC = np.nanmedian(BG, axis = (1,2))

                maxcolor = np.nanmax(Maskdata[eventtime[i][0]:eventtime[i][-1]]*eventmask[i])

                fig = plt.figure(figsize=(10,6))
                # set up subplot grid
                gridspec.GridSpec(3,3)

                # large subplot
                plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=3)
                plt.title('Masked light curve (BJD '+str(round(time[eventtime[i][0]]-time[0],2))+', RA '+str(round(Coord[0],3))+', DEC '+str(round(Coord[1],3))+')')
                plt.xlabel('Time (+'+str(time[0])+' BJD)')
                plt.ylabel('Flux')
                plt.plot(time - time[0], LC,'.', label = 'Event LC')
                plt.plot(time - time[0], BGLC,'k.', label = 'Background LC')
                plt.axvspan(time[eventtime[i][0]]-time[0]-.5/24,time[eventtime[i][-1]]-time[0]+.5/24, color = 'orange', label = 'Event duration')
                plt.axvline(time[quality[0]]-time[0],color = 'red', linestyle='dashed',label = 'Quality', alpha = 0.5)
                for j in range(quality.shape[0]-1):
                    j = j+1 
                    plt.axvline(time[quality[j]]-time[0], linestyle='dashed', color = 'red', alpha = 0.5)
                # plot Thurster firings 
                plt.axvline(time[thrusters[0]]-time[0],color = 'red',label = 'Thruster', alpha = 0.5)
                for j in range(thrusters.shape[0]-1):
                    j = j+1 
                    plt.axvline(time[thrusters[j]]-time[0],color = 'red', alpha = 0.5)
                xmin = time[eventtime[i][0]]-time[0]-(eventtime[i][-1]-eventtime[i][0])/10
                xmax = time[eventtime[i][-1]]-time[0]+(eventtime[i][-1]-eventtime[i][0])/10
                if xmin < 0:
                    xmin = 0
                if xmax > time[-1] - time[0]:
                    xmax = time[-1] - time[0]
                plt.xlim(xmin,xmax) # originally 48 for some reason
                plt.ylim(0,np.nanmax(LC[eventtime[i][0]:eventtime[i][-1]])+0.1*np.nanmax(LC[eventtime[i][0]:eventtime[i][-1]]))
                plt.legend(loc = 1)
                # small subplot 1 Reference image plot
                plt.subplot2grid((3,3), (0,2))
                plt.title('Reference')
                plt.imshow(Maskdata[Framemin,:,:], origin='lower',vmin=0,vmax = maxcolor)
                plt.colorbar()
                # small subplot 2 Event mask
                plt.subplot2grid((3,3), (1,2))
                plt.title('Mask')
                plt.imshow(eventmask[i], origin='lower')
                plt.colorbar()
                # small subplot 3 Image of event
                plt.subplot2grid((3,3), (2,2))
                plt.title('Event')
                plt.imshow(Maskdata[np.where(Maskdata*eventmask[i]==np.nanmax(Maskdata[eventtime[i][0]:eventtime[i][-1]]*eventmask[i]))[0][0],:,:], origin='lower',vmin=0,vmax = maxcolor)
                plt.colorbar()
                if eventmask.shape[0] == 1 or eventmask.shape[1] == 1:
                    plt.xticks([-0.5,0,0.5],['',0,''])
                # fit subplots and save fig
                fig.tight_layout()
                #fig.set_size_inches(w=11,h=7)
                plt.savefig(save+pixelfile.split('/')[-1].split('-')[0]+'_'+str(counter)+'.pdf', bbox_inches = 'tight');
                Result[3] += 1
                counter += 1


            Result[0] = int(pixelfile.split('ktwo')[-1].split('-')[0])
            Result[1] = (time[-1] - time[0]) - 3*len(thrusters)/48 # Last term is for removing coincident times
            Result[2] = np.nansum(Mask*1)
        else:
            Result = np.ones(5)*np.nan
    except (OSError):
        Result = np.ones(5)*-1
    return Result

In [33]:

# coding: utf-8

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.signal import convolve2d
from scipy.signal import deconvolve
from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from scipy.ndimage.filters import convolve

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

from glob import glob
import os

from tqdm import tnrange, tqdm_notebook
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

def MinThrustframe(data,thrust):
    mean = np.nanmean(data[thrust+1],axis = 0)
    std = np.nanstd((data[thrust+1] - mean), axis = (1,2))
    Framemin = np.where(std == np.nanmin(abs(std)))[0][0]
    return thrust[Framemin]+1

def DriftKiller(data,thrust):
    # The right value choice here is a bit ambiguous, though it seems that typical variations are <10.
    Drift = (abs(data[thrust+1]-data[thrust-1]) < 10)*1.0 
    Drift[Drift == 0] = np.nan
    j = 0
    for i in range(len(thrust)):
        data[j:thrust[i]] = data[j:thrust[i]]*Drift[i]
        j = thrust[i]
    return data

def FindMinFrame(data):
    # Finding the reference frame
    n_steps = 12
    std_vec = np.zeros(n_steps)
    for i in range(n_steps):
        std_vec[i] = np.nanstd(data[i:-n_steps+i:n_steps,:,:] - data[i+n_steps*80,:,:])
    Framemin = np.where(std_vec==np.nanmin(std_vec))[0][0]
    return Framemin

def ObjectMask(datacube,Framemin):
    # Make a mask of the target object, using the reference frame 
    Mask = datacube[Framemin,:,:]/(np.nanmedian(datacube[Framemin,:,:])+np.nanstd(datacube[Framemin,:,:]))
    Mask[Mask>=1] = np.nan
    Mask[Mask<1] = 1
    # Generate a second mask from remainder of the first. This grabs the fainter pixels around known sources
    Maskv2 = datacube[Framemin,:,:]*Mask/(np.nanmedian(datacube[Framemin,:,:]*Mask)+np.nanstd(datacube[Framemin,:,:]*Mask))
    Maskv2[Maskv2>=1] = np.nan
    Maskv2[Maskv2<1] = 1
    return Maskv2

def ThrustObjectMask(data,thrust):
    StartMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        Start = data[thrust[:3]+1]*StartMask/(np.nanmedian(data[thrust[:3]+1]*StartMask, axis = (1,2))+np.nanstd(data[thrust[:3]+1]*StartMask, axis = (1,2)))[:,None,None]
        Start = Start >= 1
        temp = (np.nansum(Start*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        StartMask = StartMask*temp


    EndMask = np.ones((data.shape[1],data.shape[2]))
    for i in range(2):
        End = data[thrust[-3:]+1]*EndMask/(np.nanmedian(data[thrust[-3:]+1]*EndMask, axis = (1,2))+np.nanstd(data[thrust[-3:]+1]*EndMask, axis = (1,2)))[:,None,None]
        End = End >= 1
        temp = (np.nansum(End*1, axis = 0) >=1)*1.0
        temp[temp>=1] = np.nan
        temp[temp<1] = 1
        EndMask = EndMask*temp
    
        
    Mask = np.nansum([np.ma.masked_invalid(StartMask).mask,np.ma.masked_invalid(EndMask).mask],axis=(0))*1.0
    Mask[Mask!=2] = 1
    Mask[Mask==2] = np.nan
    return Mask



def EventSplitter(events,Times,Masks,framemask):
    Events = []
    times = []
    mask = []
    for i in range(len(events)):
        # Check if there are multiple transients
        Coincident = Masks[events[i]]*framemask[events[i]]*1
        positions = np.where(Coincident == 1)
        if len(positions[0]) >1:
            for p in range(len(positions[0])):
                eventmask = np.zeros((Masks.shape[1],Masks.shape[2]))
                eventmask[positions[0][p],positions[1][p]] = 1
                eventmask = convolve(eventmask,np.ones((3,3)),mode='constant', cval=0.0)
                Similar = np.where((Masks[Times[i][0]:,:,:]*eventmask == eventmask).all(axis=(1,2)))[0]
                
                if len((np.diff(Similar)<5)) > 1:
                    
                    if len(np.where((np.diff(Similar)<5) == False)[0]) > 0:
                        SimEnd = np.where((np.diff(Similar)<5) == False)[0][0] 
                    else:
                        SimEnd = -1
                else:
                    SimEnd = 0
                #if len(np.where((np.diff(Similar)<5) == False)[0]) < 1:
                #    SimEnd = -1
                #else:
                #    SimEnd = np.where((np.diff(Similar)<5) == False)[0][0] - 1
                Similar = Similar[:SimEnd]
                if len(Similar) > 1:
                    timerange = [Similar[0]+Times[i][0]-1,Similar[-1]+Times[i][0]+1]
                    if len(timerange) > 1:
                        Events.append(events[i])
                        times.append(timerange)
                        mask.append(eventmask)
                
        else:
            Events.append(events[i])
            times.append(Times[i])
            mask.append(Masks[events[i]])
            

    return Events, times, mask

def Asteroid_fitter(Mask,Time,Data, plot = False):
    lc = np.nansum(Data*Mask,axis=(1,2))
    middle = np.where(np.nanmax(lc[Time[0]-1:Time[-1]+1]) == lc)[0][0]
    x = np.arange(middle-2,middle+2+1,1)
    x2 = np.arange(0,len(x),1)
    y = lc[[np.arange(middle-2,middle+2+1,1)]]
    p1, residual, _, _, _ = np.polyfit(x,y,2, full = True)
    p2 = np.poly1d(p1)
    AvLeft = np.nansum(abs(lc[Time[0]:Time[-1]] - p2(np.arange(Time[0],Time[-1]))))/(Time[-1]-Time[0])
    maxpoly = np.where(np.nanmax(p2(x)) == p2(x))[0][0]
    if (AvLeft < 100) &  (abs(middle - x[maxpoly]) < 2):
        asteroid = True
        if plot == True:
            p2 = np.poly1d(p1)
            plt.figure()
            plt.plot(x,lc[x],'.')
            plt.plot(x,p2(x),'.')
            plt.ylabel('Counts')
            plt.xlabel('Time')
            plt.title('Residual = ' + str(residual))
            
    else:
        asteroid = False

        
    return asteroid 

def Smoothmax(interval,Lightcurve,qual):
    x = np.arange(interval[0],interval[1],1.)
    x[qual[interval[0]:interval[-1]]!=0] = np.nan 
    nbins = int(len(x)/5)
    y = np.copy(Lightcurve[interval[0]:interval[-1]])
    y[qual[interval[0]:interval[-1]]!=0] = np.nan
    
    if np.nansum(x) > 0:
        n, _ = np.histogram(x, bins=nbins,range=(np.nanmin(x),np.nanmax(x)))
        sy, _ = np.histogram(x, bins=nbins, weights=y,range=(np.nanmin(x),np.nanmax(x)))
        sy2, _ = np.histogram(x, bins=nbins, weights=y*y,range=(np.nanmin(x),np.nanmax(x)))
        mean = sy / n
        std = np.sqrt(sy2/n - mean*mean)

        xrange = np.linspace(np.nanmin(x),np.nanmax(x),len(x))
        y_smooth = np.interp(xrange, (_[1:] + _[:-1])/2, mean)
        y_smooh_error = np.interp(xrange, (_[1:] + _[:-1])/2, std)

        temp = np.copy(y)
        temp[y_smooh_error>10] =np.nan

        maxpos = np.where(temp == np.nanmax(temp))[0]+interval[0]
    else:
        maxpos = 0
    return maxpos

def ThrusterElim(Events,Times,Masks,Firings,Quality,qual,Data):
    temp = []
    temp2 = []
    temp3 = []
    asteroid = []
    asttime = []
    astmask = []
    for i in range(len(Events)):
        Range = Times[i][-1] - Times[i][0]
        if (Range > 0) & (Range/Data.shape[0] < 0.8) & (Times[i][0] > 5): 

            begining = Firings[(Firings >= Times[i][0]-3) & (Firings <= Times[i][0]+1)]
            if len(begining) == 0:
                begining = Quality[(Quality >= Times[i][0]-1) & (Quality <= Times[i][0]+1)]
            end = Firings[(Firings >= Times[i][-1]-1) & (Firings <= Times[i][-1]+3)]
            if len(end) == 0:
                end = Quality[(Quality >= Times[i][-1]-1) & (Quality <= Times[i][-1]+1)]
            eventthrust = Firings[(Firings >= Times[i][0]) & (Firings <= Times[i][-1])]
            
            if (~begining.any() & ~end.any()) & (len(eventthrust) < 3):
                
                if Asteroid_fitter(Masks[i],Times[i],Data):
                    asteroid.append(Events[i])
                    asttime.append(Times[i])
                    astmask.append(Masks[i])
                else:
                    temp.append(Events[i])
                    temp2.append(Times[i])
                    temp3.append(Masks[i])

            elif len(eventthrust) >= 3:

                if begining.shape[0] == 0:
                    begining = 0
                else:
                    begining = begining[0]   
                if end.shape[0] == 0:
                    end = Times[i][-1] + 10
                else:
                    end = end[0]
                LC = np.nansum(Data*Masks[i], axis = (1,2))
                maxloc = Smoothmax(Times[i],LC,qual)

                if ((maxloc > begining) & (maxloc < end)): 
                    postmean = np.nanmedian(LC[eventthrust[(qual[eventthrust+1]==0)]+1])
                    Outsidethrust = Firings[(Firings < Times[i][0]) | (Firings > Times[i][-1]+20)]
                    Outsidemean = np.nanmedian(LC[(Outsidethrust[(qual[Outsidethrust+1]==0)]+1)])
                    Outsidestd = np.nanstd(LC[(Outsidethrust[(qual[Outsidethrust+1]==0)]+1)])

                    if  postmean > Outsidemean+Outsidestd:
                        temp.append(Events[i])
                        temp2.append(Times[i])
                        temp3.append(Masks[i])


    events = np.array(temp)
    eventtime = np.array(temp2)
    eventmask = np.array(temp3)
    #print(events)
    return events, eventtime, eventmask, asteroid, asttime, astmask


def pix2coord(x,y,mywcs):
    wx, wy = mywcs.wcs_pix2world(x, y, 0)
    return np.array([float(wx), float(wy)])

def Get_gal_lat(mywcs,datacube):
    ra, dec = mywcs.wcs_pix2world(int(datacube.shape[1]/2), int(datacube.shape[2]/2), 0)
    b = SkyCoord(ra=float(ra)*u.degree, dec=float(dec)*u.degree, frame='icrs').galactic.b.degree
    return b




def K2tranPix(data, time, Qual, mywcs, Position, Time): 
    try:
        hdu = fits.open(pixelfile)
        dat = hdu[1].data
        datacube = fits.ImageHDU(hdu[1].data.field('FLUX')[:]).data
        if datacube.shape[1] > 1 and datacube.shape[2] > 1:
            #print('Run')
            thrusters = np.where((Qual == 1048576) | (Qual == 1089568) | (Qual == 1056768) | (Qual == 1064960) | (Qual == 1081376) | (Qual == 10240) | (Qual == 32768) )[0]
            quality = np.where(Qual != 0)[0]
            #calculate the reference frame
            Framemin = FindMinFrame(datacube)
            # Apply object mask to data
            Mask = ThrustObjectMask(datacube,thrusters)

            Maskdata = datacube*Mask
            #Maskdata[Maskdata<0] = 0 

            #Motion control
            #Conv = convolve2d(np.ma.masked_invalid(Mask).mask, np.ones((3,3)), mode="same")
            #Maskdata = Maskdata*(Conv < 2)

            # Make a mask for the object to use as a test to eliminate very bad pointings
            obj = np.ma.masked_invalid(Mask).mask
            objmed = np.nanmedian(datacube[thrusters+1]*obj,axis=(0))
            objstd = np.nanstd(datacube[thrusters+1]*obj,axis=(0))
            Maskdata[(np.nansum(datacube*obj,axis=(1,2)) < np.nansum(objmed-3*objstd)),:,:] = np.nan

            #overflow = ((convolve(Maskdata[(np.nansum(datacube*obj,axis=(1,2)) > np.nansum(objmed+2*objstd)),:,:],
                                  #np.ones((1,1,2)), mode='constant', cval=0.0)) >= 1)*1.0
            #overflow[overflow == 1] = np.nan
            #overflow[overflow == 0] = 1
            #Maskdata[(np.nansum(datacube*obj,axis=(1,2)) > np.nansum(objmed+2*objstd)),:,:] = overflow
            #Maskdata[Maskdata > 170000] = np.nan
            #Stdframe = np.ones(Maskdata.shape)
            framemask = np.zeros(Maskdata.shape)

            #Index = (np.nansum(datacube*obj,axis=(1,2))>np.nansum(objmed-3*objstd)) #((np.nanstd(Maskdata,axis=(1,2)) > np.nanmedian(stddist)) & ((Maskdata.shape[1]>1) & (Maskdata.shape[2]>1))) 
            #framemask[Index] = (Maskdata[Index]/(np.nanmedian(Maskdata[Index])+2*(np.nanstd(Maskdata[Index])))) >= 1
            framemask = ((Maskdata/abs(np.nanmedian(Maskdata, axis = (0))+3*(np.nanstd(Maskdata, axis = (0))))) >= 1)
            framemask[:,np.where(Maskdata > 100000)[1],np.where(Maskdata > 100000)[2]] = 0
            #Index = ((np.nanstd(Maskdata) > np.nanmedian(stddist)+np.nanstd(stddist)) & ((Maskdata.shape[1]==1) | (Maskdata.shape[2] == 1))) 

            # Identify if there is a sequence of consecutive or near consecutive frames that meet condtition 
            #Eventmask = (convolve(framemask,np.ones((5,3,3)),mode='constant', cval=0.0) >= 3)
            Real_position = np.zeros((data.shape[1],data.shape[2]))
            Real_position[Position[0],Position[1]] = 1


            Eventmask = (convolve(framemask,np.ones((1,3,3)),mode='constant', cval=0.0))*1
            Eventmask[~np.where((convolve(Eventmask,np.ones((5,1,1)),mode='constant', cval=0.0) >= 4))[0]] = 0
            Eventmask[Qual!=0,:,:] = False
            #print(np.where(np.nansum(Real_position*Eventmask,axis=(1,2))>0)[0])

            #Eventmask = DriftKiller(Eventmask*Maskdata,thrusters) > 0
            #Eventmask[np.isnan(Eventmask)] = 0
            Index = np.where(np.nansum(Eventmask*1, axis = (1,2))>0)[0]
            events = []
            eventtime = []
            while len(Index) > 1:
                #print('1')
                similar = Index[((Eventmask[Index[0]]*Eventmask[Index]) == Eventmask[Index[0]]).all(axis = (1,2))]
                
                #similar = similar[np.append(True, np.diff(similar)<5)]
                if len((np.diff(similar)<5)) > 1:
                    #print('2')
                    
                    if len(np.where((np.diff(similar)<5) == False)[0]) > 0:
                        simEnd = np.where((np.diff(similar)<5) == False)[0][0] 
                    else:
                        simEnd = -1
                else:
                    simEnd = 0

                if simEnd > 0:
                    similar = similar[:simEnd]
                else:
                    similar = np.array([similar[0]])

                if len(similar) > 1:
                    events.append(similar[0])
                    temp = [similar[0],similar[-1]]
                    eventtime.append(temp)
                    temp = []
                for number in similar:
                    if ((Eventmask[Index[0]]*Eventmask[number]) == Eventmask[number]).all():
                        Index = np.delete(Index, np.where(Index == number)[0])
            print(events)
            events, eventtime, eventmask = EventSplitter(events,eventtime,Eventmask,framemask)     
            #eventtime = np.array(eventtime)
            events = np.array(events)
            eventmask = np.array(eventmask)
            if (Real_position*eventmask).any():
                print('2')
                thing = events[np.where(np.nansum(Real_position*eventmask,axis=(1,2))>0)[0]]
                print(thing)
                #print(eventtime[thing])


            # Eliminate events that begin/end within 2 cadences of a thruster fire

            events, eventtime, eventmask, asteroid, asttime, astmask = ThrusterElim(events,eventtime,eventmask,thrusters,quality,Qual,Maskdata)
            events = np.array(events)
            eventtime = np.array(eventtime)
            eventmask = np.array(eventmask)


            temp = []
            temp2 = []
            temp3 = []
            for i in range(len(eventtime)):
                if len(eventtime[i])>0:
                    t = np.nansum(Eventmask[eventtime[i][0]:eventtime[i][-1],:,:]*1,axis=(1,2)) > 0
                    if np.sum(t)/t.shape[0] > 0.6:
                        temp.append(eventtime[i][:])
                        temp2.append(events[i])
                        temp3.append(eventmask[i])

            eventtime = np.array(temp)
            events = np.array(temp2)
            eventmask = np.array(temp3)


            temp = []
            for i in range(len(events)):
                if len(np.where(datacube[eventtime[i,0]:eventtime[i,-1]]*eventmask[i] > 100000)[0]) == 0:
                    temp.append(i)
            eventtime = eventtime[temp]
            events = events[temp]
            eventmask = eventmask[temp]



            Real_position = np.zeros((data.shape[1],data.shape[2]))
            Real_position[Position[0],Position[1]] = 1
            detection = []
            if len(events) > 0:
                Overlap = (((Real_position*eventmask) == 1).any(axis = (1, 2))) & ((time[eventtime[:,0]] - time[Time]) < 4) & (Time < eventtime[:,0])

                detection = events[Overlap]

                false_detection = len(events[~Overlap])
            else:
                false_detection = 0.

            if len(detection) > 0:
                detect = 1.
            else:
                detect = 0.
            if np.nansum(obj*Real_position) > 0:
                Masked = 1.
            else:
                Masked = 0.

        else:
            detect = -1.
            false_detection = -1.
            Masked = -1.
    except (OSError):
        detect = -1.
        false_detection = -1.
        Masked = -1.
    return detect, false_detection, Masked



In [ ]:
thing[0]

In [ ]:
np.nansum(thing[thing[:,4]==1,4])*4

In [ ]:
np.nansum(thing[thing[:,4]==1,4]*thing[thing[:,4]==1,2])*4 # Units arcsec^2days

In [ ]:
np.save('c5Run1.npy',thing)

In [ ]:
thing.shape